In [1]:
using DelimitedFiles, Plots

In [2]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`


In [3]:
using CUDA
CUDA.device!(0)

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA /home/louise/.julia/packages/CUDA/Uurn4/src/initialization.jl:70


CuDevice(0): TITAN RTX

In [4]:
T = Float32
q = 21
ctype=Symbol("amino")
typel=Symbol("bm")
muext = 0.00
muint = 2.50;

In [81]:
fam = "PF00397"

path = "/home/louise/MSA/Data/test/PF00397/"
open(path)
L = 31; 

param_file = path * "Parameters_plm_PF00397.dat"
J, H = BpAlignGpu.read_parameters(param_file, q, L, gap=0, typel=typel);

delta = 0;
file_full = path * "PF00397_full_length_sequences.fasta"
file_hmmer = path * "PF00397_full.fasta"
file_hmmer_ins = path * "PF00397_full.inse"
al = BpAlignGpu.enveloptoalign( file_full, file_hmmer, file_hmmer_ins, delta = delta, ctype = ctype);
M = length(al)

using DelimitedFiles
lambda_file = path * "Lambda_PF00397.dat"
Lambda_all = readdlm(lambda_file)
lambda_o = Lambda_all[1,:];
lambda_e = Lambda_all[2,:];

┌ Info: Assuming J i j a b and h i a format
└ @ BpAlignGpu /home/louise/MSA/BpAlignGpu.jl/src/utils.jl:37
┌ Info: Output tersors: J[a b i j] and h[a i]
└ @ BpAlignGpu /home/louise/MSA/BpAlignGpu.jl/src/utils.jl:39
┌ Info: Gap in input file 0 now in 21
└ @ BpAlignGpu /home/louise/MSA/BpAlignGpu.jl/src/utils.jl:40


In [103]:
file_seed = path * "PF00397_seed.fasta"
seq_seed = readdlm(file_seed);
Mseed = size(seq_seed,1)
names_s = [String(seq_seed[i]) for i=1:2:Mseed-1];

In [104]:
names_full = ">".*[x[1] for x in al];

In [ ]:
function cutname(s)
    n=0
    for (i,c) in pairs(s)
        if c == '/'
            n=i
        end
    end
    return s[1:n-1]
end

In [146]:
names_full_cut = cutname.(names_full);

In [167]:
i=75
idx0 = findall(names_full .== names_s[i])

Int64[]

In [168]:
idx0 = findall(names_full_cut .== cutname(names_s[i]))

Int64[]

In [169]:
names_s[i]

">Q23425_CAEEL/19-49"

In [186]:
for i in 1:length(names_s)
    idx0 = findall(names_full .== names_s[i])
    if idx0 == Int64[]
       idx0 = findall(names_full_cut .== cutname(names_s[i])) 
    end
    if length(idx0) == 1
        t = 2*(i-1)+1
        #@show idx0, names_s[i] == seq_seed[t]
        open("inds_seed_"*fam*".txt", "a") do io
             writedlm(io, [idx0 seq_seed[t] seq_seed[t+1]])
        end
    else
        #@show i,idx0        
    end

end

In [179]:
i=10
t = 2*(i-1)+1

19

In [180]:
seq_seed[t:t+1]

2-element Vector{Any}:
 ">D0NB34_PHYIT/109-138"
 "DYQKWIESFTED-GYPFYYNTNTGESSWYKP"

In [181]:
al[16444]

("D0NB34_PHYIT/111-138", "QKWIESFTEDGYPFYYNTNTGESSWYKP", "----WIESFTED-GYPFYYNTNTGESSWYKP", "----WIESFTED-GYPFYYNTNTGESSWYKP")